In [1]:
import pandas as pd
import json
from langchain_openai import ChatOpenAI
import asyncio
import nest_asyncio
from config import *
def get_question(question_path):
    try:
        df = pd.read_json(question_path)
        return df
    except FileNotFoundError:
        print(f"Error: The file {question_path} was not found.")
        return pd.DataFrame()  # 返回空 DataFrame
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
        return pd.DataFrame()


In [2]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_BASE_URL,
    model='gpt-4o-2024-08-06',
    temperature=0,
    #model='o1-mini'
)


In [4]:
q3=get_question(r'耗材\耗材1\Final_Example.json')
q2=get_question(r'耗材\耗材2\Final_Example.json')
q1=get_question(r'耗材\耗材3\Final_Example.json')
tmp_num=0
async def process_question(index, row, semaphore):
    async with semaphore:
        if index>=tmp_num and index<=256:
            # 获取 'answer' 字段的值
            answer_q1 = q1.iloc[index]['answer']
            code_q1=q1.iloc[index]['code']
            answer_q2 = q2.iloc[index]['answer']
            code_q2=q2.iloc[index]['code']
            answer_q3 = q3.iloc[index]['answer']
            code_q3=q3.iloc[index]['code']
            question=q2.iloc[index]['question']
            is_verify1=q1.iloc[index]['is_verify']
            is_verify2=q2.iloc[index]['is_verify']
            is_verify3=q3.iloc[index]['is_verify']
            # 如果 answer_q1 或 answer_q2 是列表或字典，先将其转换为字符串
            if isinstance(answer_q1, (dict, list)):
                answer_q1 = str(answer_q1)
            if isinstance(answer_q2, (dict, list)):
                answer_q2 = str(answer_q2)
            if isinstance(answer_q3, (dict, list)):
                answer_q3 = str(answer_q3)
            res=await llm.ainvoke(f"""
            你是一个图论专家，精通精通基础图论、图统计学习和图嵌入三个方面的知识
            你现在需要根据问题帮我在答案1和答案2答案3之间选出更正确的选项
            需要少数服从多数，即当有两个答案比较相似时则在这两个之间选出更正确的异相
            问题:{question}
            答案1:{answer_q1}
            答案1代码:{code_q1}
            答案2:{answer_q2}
            答案2代码:{code_q2}
            答案3:{answer_q3}
            答案3代码:{code_q3}
            你必须只能输出1或者2或者3来选择问题1或者问题2或者问题3
    """)
            res=res.content
            id=q2.iloc[index]['ID']
            print(f'问题{id}的答案是{res}')
            if '1' in res:
                q2.loc[index, 'answer'] = answer_q1
                q2.loc[index, 'code']=code_q1
                q2.loc[index,'is_verify']=is_verify1
                tmp_str=q2.iloc[index]['answer']
            elif '3' in res:
                q2.loc[index, 'answer'] = answer_q3
                q2.loc[index, 'code']=code_q3
                q2.loc[index,'is_verify']=is_verify3
                tmp_str=q2.iloc[index]['answer']
            data_list = q2.to_dict(orient='records')
            json_str = json.dumps(data_list, separators=(',', ':'),ensure_ascii=False)
            formatted_json_str = json_str.replace('},{', '},\n{')

            with open('ans.json', 'w', encoding='utf-8') as f:
                f.write(formatted_json_str)
                f.close()

In [5]:
async def main():
    semaphore = asyncio.Semaphore(20)  # 限制最多5个任务同时运行
    tasks = [process_question(index, row, semaphore) for index, row in q1.iterrows()]
    await asyncio.gather(*tasks)

In [ ]:

# 允许在已有的事件循环中再次运行事件循环
nest_asyncio.apply()
# 替换asyncio.run(main())为以下代码
async def main_wrapper():
    await main()

# 获取当前的事件循环并运行
loop = asyncio.get_event_loop()
loop.run_until_complete(main_wrapper())